In [ ]:
!pip install ultralytics gdown tqdm albumentations
!git clone https://github.com/n24q02m/VehicleDetection.git

In [ ]:
!cp /kaggle/working/VehicleDetection/patches/check.py /usr/local/lib/python3.*/dist-packages/ultralytics/utils/check.py
!cp /kaggle/working/VehicleDetection/patches/task.py /usr/local/lib/python3.*/dist-packages/ultralytics/nn/tasks.py
!cp /kaggle/working/VehicleDetection/patches/modules_init.py /usr/local/lib/python3.*/dist-packages/ultralytics/nn/modules/__init__.py
!cp /kaggle/working/VehicleDetection/patches/conv.py /usr/local/lib/python3.*/dist-packages/ultralytics/nn/modules/conv.py

In [ ]:
from ultralytics import YOLO
from VehicleDetection.src.training.distillation import DistillationTrainer
from VehicleDetection.src.training.train import read_augmentation_parameters

In [ ]:
# Configure paths for Kaggle
model_name = "/kaggle/working/VehicleDetection/models/custom-yolov8m-ghost-p2.yaml"
data_dir = "/kaggle/working/VehicleDetection/data/soict-hackathon-2024_dataset"
train_project = "/kaggle/working/VehicleDetection/runs"
train_name = "distillation_custom-yolov8m-ghost-p2"

# Read augmentation parameters
augmentation_params = read_augmentation_parameters(
    "/kaggle/working/VehicleDetection/runs/mosaic_erasing.txt"
)

# Initialize student model
model = YOLO(model_name)

# Training parameters optimized for Kaggle
train_params = {
    "data": f"{data_dir}/data.yaml",
    "epochs": 600,
    "time": 8,  # 8 hours max for Kaggle session
    "batch": 0.9,  # Use 90% of GPU memory
    "cache": True,  # Cache in RAM
    "device": 0,
    "project": train_project,
    "name": train_name,
    "exist_ok": True,
    "optimizer": "auto",
    "seed": 42,
    "cos_lr": True,
    "fraction": 0.1,
    "multi_scale": True,
    "augment": True,
    "label_smoothing": 0.1,
    **augmentation_params,
}

# Start training with distillation
model.train(trainer=DistillationTrainer, **train_params)